Quick Drawing is a kaggle challenge for classifying the sketch pictures. The total number of category is 340 and 
there are 50M pictures in the dataset. Each catogory has 25,000 pictures. The difficulty is the drawing might be 
incomplete or the drawing does not match the label. We are trying to develop a CNN to solve the issue.   

According to the challenge, there are 4 kinds of dataset. 

    - Raw data (.ndjson)
    - Simplified drawings (.ndjson)
    - Binary data (.bin)
    - Bitmap data (.npy)
Raw data and Simplified drawings are provided in a vector information of strokes. Bitmap data is rendered into 
28 * 28 grayscale bitmap in the numpy format.  

Maybe we could consider the raw data to improve the performance since it inlcudes the sequence information.


In [1]:
import torch
import time
import torch
import os
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
from torch.utils import data
import QDdata as QD
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
import resnet as RN
from torchvision import transforms

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 300

print('using device:', device)


using device: cuda


# Data Augmentation

In [3]:
trans = transforms.Compose([transforms.RandomHorizontalFlip(0.5), transforms.ToTensor()])

# Resnet50 96X96

In [4]:
# The following codes comes from cs231N.
from average_precision import mapk
def mean_ap_torch(y_val,y_pred_score,k):
    y_pred = [torch.sort(c,descending=True)[::-1][1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def mean_ap_3(y_val,y_pred_score,k):
    y_pred = [np.argsort(c)[::-1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def check_accuracy(loader, model):
    
    model.eval()  # set model to evaluation mode
    
    meanAP = []
    meanAP3 = []
    with torch.no_grad():
        for i ,(x, y) in enumerate(loader):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
            if i > 300:
                break
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        

        if (m3 > QDtrain["mAP3"] + 1) and (m3 > 20):
            if os.path.exists(QDtrain["file"]):
                os.remove(QDtrain["file"])
            QDtrain["file"] = str("model" + QDtrain["prefix"] + str(m3))
            torch.save(model.state_dict(),QDtrain["file"])
            QDtrain["mAP3"] = m3
            QDtrain["mAP"] = m1
        print('Got correct (%.2f,%.2f)' % (m1,m3 ))
        
        
def train(model, optimizer, epochs=1):
    """
    Train a model on quickdrawing.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loader_val = QD.QDloadData(no=10)
    loader_val = DataLoader(dataset=loader_val, batch_size=128,shuffle=True)
    
    for e in range(epochs):
        
        for i in range(4):
            train_dataset = QD.QDloadData(no=i,transforms = trans)          
            train_loader = DataLoader(dataset=train_dataset, batch_size=128,shuffle=True)
            #loader_val = train_loader
            t1 = time.time()
            for t, (x, y) in enumerate(train_loader):
                model.train()  # put model to training mode
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=torch.long)

                scores = model(x)
                loss = F.cross_entropy(scores, y)

                # Zero out all of the gradients for the variables which the optimizer
                # will update.
                optimizer.zero_grad()

                # This is the backwards pass: compute the gradient of the loss with
                # respect to each  parameter of the model.
                loss.backward()

                # Actually update the parameters of the model using the gradients
                # computed by the backwards pass.
                optimizer.step()
                t2 = time.time()

                #print(e,t,t2-t1)
                if t % print_every == 0:
                    print('Epoch %d,Iteration %d, loss = %.4f time %.4f' % (e,t, loss.item(),t2-t1))
                    check_accuracy(loader_val, model)
                    t1 = time.time()

In [5]:
model = RN.resnet50(num_classes=40)
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(),weight_decay = 1e-5,lr=learning_rate)
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"resnet50"}

In [ ]:
train(model, optimizer, epochs=5)

data10.npy label10.npy
Total number of items: 120000
data0.npy label0.npy
Total number of items: 120000
Epoch 0,Iteration 0, loss = 3.9519 time 0.1754
Got correct (2.42,4.43)
Epoch 0,Iteration 300, loss = 1.4472 time 47.7834
Got correct (53.18,63.93)
Epoch 0,Iteration 600, loss = 1.1170 time 46.9895


1. batch size = 64. count =  340 * 10240.
   Iteration 29400, loss = 0.5610
   Got 64 correct (80.22,85.62)
2. batch size =128 count = 340 *10240 * 2
    Iteration 37800, loss = 0.9535
    Got 128 correct (77.65,83.64)
3. batch size =128 count = 340 *10240 * 2 
    train on 28 * 28 drawing transfered from stroke
    Iteration 45900, loss = 0.9149
    Got correct (57.55,65.98)

# Baseline 28X28

In [ ]:
train_dataset28 = QD.QDDetection(count = 40 * 3000*9 ,images_category = 3000*9,transforms = trans)
loader_dataset28 = QD.QDDetection(count = 40 * 512,start = 3000*9, images_category = 3000)
train_loader28 = DataLoader(dataset=train_dataset28, batch_size=128, num_workers=2, shuffle=True)
loader_val28 = DataLoader(dataset=loader_dataset28, batch_size=128,shuffle=True)

def train28(model, optimizer, epochs=1):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        t1 = time.time()
        for t, (x, y) in enumerate(train_loader28):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            t2 = time.time()
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f time:%.4f' % (t, loss.item(),t2 - t1))
                check_accuracy(loader_val28, model)
                t1 = time.time()


learning_rate28 = 1e-3

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

    
model28 = nn.Sequential(
    nn.Conv2d(1,64,3,padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64,64,3,padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64,128,3,padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128,256,3,padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.AvgPool2d(4),
    Flatten(),
    nn.Linear(12544, 40),
)

# you can use Nesterov momentum in optim.SGD
optimizer28 = optim.Adam(model28.parameters(), lr=learning_rate28)

QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"base28"}
train28(model28, optimizer28, epochs=5)

# Baseline model (96X96)

In [ ]:
base_learning_rate = 1e-4

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

    
base_model = nn.Sequential(
    nn.Conv2d(1,64,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.Conv2d(64,128,3,stride = 2,padding=1),
    nn.ReLU(),
    nn.Conv2d(128,128,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 1),
    nn.Conv2d(128,256,3,stride = 2,padding=1),
    nn.ReLU(),
    nn.Conv2d(256,256,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 1),
    Flatten(),
    nn.Linear(9216,4096),
    nn.ReLU(),
    nn.Linear(4096,40)
)


base_optimizer = optim.Adam(base_model.parameters(),lr=base_learning_rate)
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"base96"}
train(base_model, base_optimizer, epochs=5)

# Sketch-A-Net model(256 X 256)
* The following code is based on Sketch-A-Net that Beats Humans.

In [ ]:
SN_learning_rate = 1e-3

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

    
SN_model = nn.Sequential(
    nn.Conv2d(1,64,15,stride = 3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 0),
    
    nn.Conv2d(35,128,5,stride = 1,padding=0),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 0),
    
    nn.Conv2d(15,256,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.Conv2d(15,256,3,stride = 1,padding=1),
    nn.ReLU(),    
    nn.Conv2d(15,256,3,stride = 1,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3,stride = 2,padding = 0),

    Flatten(),
    nn.Linear(256*7*7,512),
    nn.ReLU(),
    nn.Dropout(),
    
    nn.Linear(512,512),
    nn.ReLU(),
    nn.Dropout(),
    
    nn.Linear(512,40)
)


SN_optimizer = optim.Adam(SN_model.parameters(),weight_decay = 1e-5,lr=SN_learning_rate)
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"sketchanet"}
train(SN_model, SN_optimizer, epochs=5)

# Run test samples

In [ ]:
def evaluate_test(test_data, model,y):
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        meanAP = []
        meanAP3 = []
        for i ,x in enumerate(test_data):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            #y = y_label.to(device=device, dtype=torch.long)
            y_pred = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        return (m1,m3)

model = base_model.to(device=device)
checkpoint = torch.load("model_parameter99.3972598.87986")
model.load_state_dict(checkpoint)

class_name = QD.QDloadtest(name = "./data1.npy")
class_name = class_name.class_names

class_accuracy = []

for i,name in enumerate(tqdm(class_name)):
    test_data = QD.QDloadtest(name=os.path.join("../cs230/pic96",name+".npz"),image_size=(96,96))
    test_loader = DataLoader(dataset=test_data, batch_size=128, num_workers=2, shuffle=False)
    y = np.ones(len(test_data))*i
    m1,m3 = evaluate_test(test_loader,model,y)
    print(name,m1,m3)
    class_accuracy.append((name,str(m1),str(m3)))
    
with open('class_accuracy.csv',"w+") as fp:
    writer = csv.writer(fp, delimiter=',')
    writer.writerow(["class", "meanAP1", "meanAP3"])  # write header
    writer.writerows(class_accuracy)


1) 96*96, resnet50, epoch 10, GSD. 75.2%
2) 96*96, resnet50, epoch 10, GSD  79.9% model_parametertensor(91.6667)
3) 96*96, resnet50, epoch 10, Adam model_resnet_92.317_90


* convert simplified test dataset to pixel drawsing

Check the accuracy based on the class.

Create test_simplified.
load compressed(96*96) 1.968482494354248
load non compressed(96*96)  0.31299567222595215
load non compressed(28*28) 0.04715704917907715


In [ ]:
for i in range(10,11):
    data = QD.QDcreateData()
    data.create(start=i*3000,dir_name="../cs230/pic96")
    del data